In [1]:
%%time
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry import Registry
import pandas as pd, re, json

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
CPU times: user 1.41 s, sys: 759 ms, total: 2.17 s
Wall time: 8.25 s


In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [3]:
session = get_session('DIAMONDS',"fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [4]:
import requests
exp_id="aaaawwwwweeeer"#"600b9f90-f431-4590-83c7-f38fe138dedc"
project_id="fd0cf79c-8118-43a7-8f0b-059e8f78227a"
executions_url = f"http://monitor-backend-service/monitor/api/v4/executions/search?objectId={exp_id}&projectId={project_id}&jobType=EXPERIMENT&profile=DETAILED"

temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "68bce134-6d7a-4e24-9591-86266438acf9"
}

In [5]:
response = requests.get(executions_url, headers=temp_head)
print(response.json())
res = response.json()

{'totalRecordCount': 32, 'data': [{'jobinstanceId': 109574, 'objectId': 'aaaawwwwweeeer', 'projectId': 'fd0cf79c-8118-43a7-8f0b-059e8f78227a', 'projectName': None, 'jobObjectName': 'exp_001', 'jobScheduleId': None, 'jobInstanceInputParameter': '{}', 'jobType': 'EXPERIMENT', 'jobStatus': 'DIDNOTRUN', 'finalStatus': 'DIDNOTRUN', 'jobCreatedby': 'aditya1.singh2@fosfor.com', 'jobCreatedts': '2024-07-09T09:07:50.960+00:00', 'jobUpdatedby': 'aditya1.singh2@fosfor.com', 'jobUpdatedts': '2024-07-09T09:07:50.960+00:00', 'applicationId': '', 'executionId': None, 'runID': '109574', 'versionNumber': None, 'runEngine': 'SNOWFLAKE', 'attemptId': 109678, 'workflowId': None, 'jobConfig': '{"snowflake_query": CALL experiment_stored_proc(\'{"monitor_run_id": "MONITORRUNID", "experiment_id": "aaaawwwwweeeer", "experiment_name": "exp_001", "stored_procedure": "run_experiment", "algorithm_category": "Snowparkml", "algorithm_type": "Classification", "dataset_name": "diamonds_dataset", "target_column": "qual

In [6]:
delete_run_url = f"http://monitor-backend-service/monitor/api/v1/delete/109801"

In [7]:
response = requests.delete(delete_run_url, headers=temp_head)
print(response.json())
res = response.json()

{'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource api/v1/delete/109801.', 'instance': '/monitor/api/v1/delete/109801'}
